In [ ]:
# !pip install twder
import twder
import pandas as pd
from ipywidgets import interact, widgets

In [11]:
def get_rate(yuan):
    data = twder.now(yuan)
    df = pd.DataFrame(data)
    df.index = ['時間', '現金買入', '現金賣出', '即期買入', '即期賣出']
    result_dict = df.to_dict()
    return result_dict

# List of currencies
yuan=['USD', 'HKD', 'GBP', 'AUD', 'CAD', 'SGD', 'CHF', 'JPY', 'ZAR', 'SEK', 'NZD', 'THB', 'PHP', 'IDR', 'EUR', 'KRW', 'VND', 'MYR', 'CNY']

# Create a dropdown widget
currency_dropdown = widgets.Dropdown(
    options=yuan,
    description='Select Currency:',
    disabled=False,
)

# Use interact to create an interactive widget
interact(get_rate, yuan=currency_dropdown);

interactive(children=(Dropdown(description='Select Currency:', options=('USD', 'HKD', 'GBP', 'AUD', 'CAD', 'SG…

In [2]:
# Download this first: https://googlechromelabs.github.io/chrome-for-testing/
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pyautogui
import time

# Notice :  the sleep time don't sub it.(if the py always failed, the time could add 1sec)

# spend 2mins 50secs to crawler code  (so long time,but the data are too much to crawl)

# Suggetion: Need to store the data in a file (using pandas to save as a file)
#            If you need to read the data, read the file
#            If you need to update the data, call this function

# two-dimensional list ->[index][0=name,1=price,2=reviews(ignore...some will be None)]
jellycat_data_store1 = list()
jellycat_data_store2 = list()
jellycat_data_store3 = list()
jellycat_data_store4 = list()

# two-dimensional list ->[index][0=name,1=price]
selfridge_data_store1 = list()
selfridge_data_store2 = list()

# two-dimensional list ->[index][0=name,1=price]
campusgifts_data_store1 = list()
campusgifts_data_store2 = list()
campusgifts_data_store3 = list()


def jellycat_webSelenium():
    # set webdriver service
    service = Service(executable_path="./chromedriver.exe")
    # set webdriver option
    option = webdriver.ChromeOptions()
    # set web always opens
    option.add_experimental_option("detach", True)
    # error notion skip
    option.add_experimental_option("excludeSwitches", ["enable-logging"])

    driver = webdriver.Chrome(service=service, options=option)
    driver.get("https://www.jellycat.com/eu/all-soft-toys/?stock_text=in%20stock&page")  # URL
    time.sleep(2)

    data_index = driver.find_element(
        By.CSS_SELECTOR,
        "#productDataOnPagex",
    )
    # count the max-page
    get_index = int(data_index.get_attribute("data-nq-maxpages"))

    # rolling the web to load the dynamic data -> need to wait
    for num in range(1, get_index + 1, 1):
        # slide-down the web 10000 pixel-sized
        driver.execute_script("window.scrollBy(0,10000)")
        time.sleep(1)

    for i in range(1, get_index + 1, 1):
        data_index = driver.find_element(
            By.XPATH,
            '//*[@id="productDataOnPagex"]/div[' + str(i) + "]",
        )
        pr = data_index.find_elements(By.CLASS_NAME, "listing-details")

        # store the data to variable
        for j in pr:
            if i > 0 and 10 >= i:
                jellycat_data_store1.append(j.text.split("\n"))
            elif i > 10 and 20 >= i:
                jellycat_data_store2.append(j.text.split("\n"))
            elif i > 20 and 30 >= i:
                jellycat_data_store3.append(j.text.split("\n"))
            else:
                jellycat_data_store4.append(j.text.split("\n"))
    driver.close()


def selfridge_webSelenium():
    service = Service(executable_path="./chromedriver.exe")
    option = webdriver.ChromeOptions()
    option.add_experimental_option("detach", True)
    option.add_experimental_option("excludeSwitches", ["enable-logging"])
    for t in range(1, 7, 1):
        driver = webdriver.Chrome(service=service, options=option)
        driver.get("https://www.selfridges.com/TW/en/cat/jellycat/?pn=" + str(t))
        time.sleep(2)

        css_path = "#content"
        data = driver.find_element(By.CSS_SELECTOR, css_path)
        name = data.find_elements(By.CLASS_NAME, "c-prod-card__cta-box-description")
        price = data.find_elements(By.CLASS_NAME, "c-prod-card__cta-box-price")

        # store the data to variable
        for i in range(0, len(name), 1):
            if t > 0 and 3 >= t:
                selfridge_data_store1.append([name[i].text, price[i].text])
            elif t > 3 and 7 >= t:
                selfridge_data_store2.append([name[i].text, price[i].text])
        driver.close()


def campusgifts_webdriver():
    service = Service(executable_path="./chromedriver.exe")
    option = webdriver.ChromeOptions()
    option.add_experimental_option("detach", True)
    option.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(service=service, options=option)
    driver.get(
        "https://www.campusgifts.co.uk/gifts-by-brand/jellycat/browse-all?product_list_limit=96"
    )
    time.sleep(0.25)
    data_index = driver.find_elements(
        By.CSS_SELECTOR, "#amasty-shopby-product-list > div:nth-of-type(2) > ol >li"
    )

    for data in data_index:
        try:
            name = data.find_element(By.CLASS_NAME, "product-item-link")
            try:
                price = data.find_element(By.CLASS_NAME, "old-price")
            except:
                continue
            try:
                price = data.find_element(By.CLASS_NAME, "special-price")
            except:
                continue
        except:
            continue

    for i in range(2, 11, 1):
        pyautogui.hotkey("ctrl", "t", interval=0.1)
        driver.switch_to.window(driver.window_handles[1])
        driver.get(
            "https://www.campusgifts.co.uk/gifts-by-brand/jellycat/browse-all?p="
            + str(i)
            + "&product_list_limit=96"
        )
        driver.switch_to.window(driver.window_handles[0])
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(2)
        data_index = driver.find_elements(
            By.CSS_SELECTOR, "#amasty-shopby-product-list > div:nth-of-type(2) > ol >li"
        )

        for data in data_index:
            try:
                name = data.find_element(By.CLASS_NAME, "product-item-link")
                try:
                    price = data.find_element(By.CLASS_NAME, "old-price")
                except:
                    continue
                try:
                    price = data.find_element(By.CLASS_NAME, "special-price")
                except:
                    continue
            except:
                continue

            # store the data to variable
            if i > 0 and 4 >= i:
                campusgifts_data_store1.append([name.text, price.text])
            elif i > 4 and 8 >= i:
                campusgifts_data_store2.append([name.text, price.text])
            elif i > 8 and 12 > i:
                campusgifts_data_store3.append([name.text, price.text])

    driver.close()


jellycat_webSelenium()
# selfridge_webSelenium()
# campusgifts_webdriver()




In [3]:
def format_data(data):
    parsed_data = {}
    for item in data:
        parsed_data[item[0]] = item[1]
    return parsed_data

# Use the function
formatted_jellycat_data_store1 = format_data(jellycat_data_store1)
formatted_jellycat_data_store2 = format_data(jellycat_data_store2)
formatted_jellycat_data_store3 = format_data(jellycat_data_store3)
formatted_jellycat_data_store4 = format_data(jellycat_data_store4)

combined_jellycat_data = []
combined_jellycat_data.extend(jellycat_data_store1)
combined_jellycat_data.extend(jellycat_data_store2)
combined_jellycat_data.extend(jellycat_data_store3)
combined_jellycat_data.extend(jellycat_data_store4)



formatted_selfridge_data_store1 = format_data(selfridge_data_store1)
formatted_selfridge_data_store2 = format_data(selfridge_data_store2)

combined_selfridge_data = []
combined_selfridge_data.extend(selfridge_data_store1)
combined_selfridge_data.extend(selfridge_data_store2)

formatted_campusgifts_data_store1 = format_data(campusgifts_data_store1)
formatted_campusgifts_data_store2 = format_data(campusgifts_data_store2)
formatted_campusgifts_data_store3 = format_data(campusgifts_data_store3)

combined_campusgifts_data = []
combined_campusgifts_data.extend(campusgifts_data_store1)
combined_campusgifts_data.extend(campusgifts_data_store2)
combined_campusgifts_data.extend(campusgifts_data_store3)

print("Jellycat: ")
print(combined_jellycat_data)

print("Campusgifts: ")
print(combined_campusgifts_data)

print("Selfridge: ")
print(combined_selfridge_data)

Jellycat: 
[['Amuseable Nordic Spruce Christmas Tree', '€60.00', '(5)'], ['Jolly Gingerbread Fred Huge', '€75.00'], ['Amuseable Present', '€35.00', '(2)'], ['Amuseable Blue Spruce Christmas Tree', '€50.00', '(3)'], ['Amuseable Mistletoe', '€25.00'], ['Ronnie Rockhopper Penguin', '€30.00', '(14)'], ['Maurice Macaroni Penguin', '€35.00', '(10)'], ['Smudge Rabbit', '€37.00', '(12)'], ['Bashful Beige Bunny', '€29.00', '(142)'], ['Bartholomew Bear with Personalised Red Jumper', '€39.50'], ['Bartholomew Bear', '€30.00', '(217)'], ['Bartholomew Bear Bedtime', '€42.00', '(20)'], ['Bashful Christmas Bunny', '€32.00', '(9)'], ['Sammie Snowman', '€40.00', '(2)'], ['Amuseable Mince Pie', '€25.00', '(4)'], ['Amuseable Yule Log', '€30.00', '(9)'], ['Amuseable Hot Chocolate', '€32.00', '(7)'], ['Amuseable Croissant Bag Charm', '€25.00', '(2)'], ['Amuseable Croissant Bag', '€40.00'], ['Christmas Grizzo', '€45.00', '(5)'], ['Santa Ricky Rain Frog', '€40.00', '(9)'], ['Otto’s Snowy Christmas Book', '€17

In [29]:

EUR = float((get_rate('EUR'))[0]['現金賣出'])
print("EUR: ")
print(EUR)
GBP = float((get_rate('GBP'))[0]['現金賣出'])
print("GBP: ")
print(GBP)


EUR: 
34.34
GBP: 
40.31


In [65]:
import copy
#報價系統2.0
#↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓

def modify_price(price):
    integer_part = int(price)

    # 如果個位數和十位數的組合小於50，將十位數設為8
    # 否則，進位到下一個百位，將十位數設為2
    if integer_part % 100 < 50:
        modified_integer = (integer_part // 100) * 100 + 80
    else:
        modified_integer = ((integer_part // 100) + 1) * 100 + 20

    return modified_integer

jellycat_prices = copy.deepcopy(combined_jellycat_data)
jellycat_costs = copy.deepcopy(combined_jellycat_data)
# 計算報價
def compute_jellycat_price(jellycat_price):
    for qutation in jellycat_price:
        price_str = qutation[1].replace('€', '')  # 去除價格字串中的 '€' 符號
        price = float(price_str)  # 將去除 '€' 符號後的價格轉換為浮點數
        product_name = qutation[0]
        
        # 根據價格和產品名稱應用不同的規則
        if price < 40:
            new_price = price * 1.15 * EUR
        elif price > 40 and price <= 100:
            new_price = price * 1.1 * EUR
        else:
            new_price = price * 1.05 * EUR

        # 如果產品名稱中包含 'bag'，應用額外規則
        if 'bag' in product_name.lower():
            new_price *= 1.2


        modified_price = modify_price(new_price)

        qutation[1] = modified_price


compute_jellycat_price(jellycat_prices)

#代購比價系統2.0
#↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
#jellycat官網成本價
def compute_jellycat_cost(jellycat_cost):
    for jelly_cost in jellycat_cost:
        price_str = jelly_cost[1].replace('€', '')  # 去除價格字串中的 '€' 符號
        price = float(price_str)  # 將去除 '€' 符號後的價格轉換為浮點數

        # 將價格乘以0.95
        new_price = price* 0.95 * EUR

        jelly_cost[1] = new_price
        jelly_cost = jelly_cost[:2]

        
compute_jellycat_cost(jellycat_costs)

def compute_selfridge_cost(selfridge_costs):
    #selfridge成本價
    for selfridge_cost in selfridge_costs:
        price_str = selfridge_cost[1].replace('$', '').replace(',', '')
        price = float(price_str)

        # 將價格乘以0.9
        new_price = price * 0.9

        selfridge_cost[1] = new_price

        
selfridge_costs = copy.deepcopy(combined_selfridge_data)
compute_selfridge_cost(selfridge_costs)

def compute_campusgifts_cost(campusgifts_costs):
    #Campusgifts成本價
    for campusgifts_cost in campusgifts_costs:

        price_str = campusgifts_cost[1].replace('£', '').replace(',', '')
        price = float(price_str)

        # 將價格乘以1.1
        new_price = price * 1.1 * GBP

        campusgifts_cost[1] = new_price

        
campusgifts_costs = copy.deepcopy(combined_campusgifts_data)
compute_campusgifts_cost(campusgifts_costs)

print(jellycat_prices)
print(jellycat_costs)
print(selfridge_costs)
print(campusgifts_costs)




[['Amuseable Nordic Spruce Christmas Tree', 2320, '(5)'], ['Jolly Gingerbread Fred Huge', 2880], ['Amuseable Present', 1420, '(2)'], ['Amuseable Blue Spruce Christmas Tree', 1920, '(3)'], ['Amuseable Mistletoe', 1020], ['Ronnie Rockhopper Penguin', 1220, '(14)'], ['Maurice Macaroni Penguin', 1420, '(10)'], ['Smudge Rabbit', 1520, '(12)'], ['Bashful Beige Bunny', 1180, '(142)'], ['Bartholomew Bear with Personalised Red Jumper', 1620], ['Bartholomew Bear', 1220, '(217)'], ['Bartholomew Bear Bedtime', 1620, '(20)'], ['Bashful Christmas Bunny', 1320, '(9)'], ['Sammie Snowman', 1480, '(2)'], ['Amuseable Mince Pie', 1020, '(4)'], ['Amuseable Yule Log', 1220, '(9)'], ['Amuseable Hot Chocolate', 1320, '(7)'], ['Amuseable Croissant Bag Charm', 1220, '(2)'], ['Amuseable Croissant Bag', 1780], ['Christmas Grizzo', 1720, '(5)'], ['Santa Ricky Rain Frog', 1480, '(9)'], ['Otto’s Snowy Christmas Book', 720], ['Winter Warmer Otto Sausage Dog', 1480, '(3)'], ['Winter Warmer Pippa Black Labrador', 1480]

In [4]:
def parse2dict(data):
    parsed_data = {}
    for item in data:
        parsed_data[item[0]] = item[1]
    return parsed_data

jellycat_prices_dict = parse2dict(jellycat_prices)
jellycat_costs_dict = parse2dict(jellycat_costs)
selfridge_costs_dict = parse2dict(selfridge_costs)
campusgifts_costs_dict = parse2dict(campusgifts_costs)

NameError: name 'jellycat_prices' is not defined

In [5]:
#代購價與成本價間的比價
def compare_prices(quotation, jelly_cost, selfridge_cost, campusgifts_cost):
    differenciate_result = {}

    if quotation[1] == None: 
        return differenciate_result
    else :
        q_price = quotation[1]
    
    if jelly_cost[1] == None: 
        j_diff = -9999
    else:
        j_diff = quotation[1] - jelly_cost[1]
        
    if selfridge_cost[1] == None:
        s_diff = -9999
    else:
        s_diff = quotation[1] - selfridge_cost[1]
        
    if campusgifts_cost[1] == None:
        c_diff = -9999
    else:
        c_diff = quotation[1] - campusgifts_cost[1]
        
    # 將結果儲存到字典中
    differenciate_result[quotation[0]] = {'quotation_price': q_price, 'jelly_cost_diff': j_diff, 'selfridge_cost_diff': s_diff, 'campusgifts_cost_diff': c_diff}

    return differenciate_result

# Example of using the function
name = 'Amuseable Nordic Spruce Christmas Tree'
quotation = [name, jellycat_prices_dict.get(name)]
jelly_cost = [name, jellycat_costs_dict.get(name)]
selfridge_cost = [name, selfridge_costs_dict.get(name)]
campusgifts_cost = [name, campusgifts_costs_dict.get(name)]

differenciate_result = compare_prices(quotation, jelly_cost, selfridge_cost, campusgifts_cost)
print(differenciate_result)

NameError: name 'jellycat_prices_dict' is not defined